In [57]:
pip install ydata-profiling

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for ydata-profiling from https://files.pythonhosted.org/packages/09/98/5a5fbfd48bf23acd3ceed5114d073b877965816304078151fefbf86d8458/ydata_profiling-4.5.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pydantic<2,>=1.8.1 from https://files.pythonhosted.org/packages/24/5c/7d600cf0c0c4fa577c33bafd4a7ee567bb19552dbd093b0d4c756ea6cc13/pydantic-1.10.12-cp39-cp39-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 4.1 MB/s eta 0:00:00
  Obtaining dependency information for PyYAML<6.1,>=5.0.0 from https://files.pythonhosted.org/packages/0e/88/21b2f16cb2123c1e9375f2c93486e35fdc86e63f02e274f0e99c589ef153/PyYAML-6.0.1-cp39-cp39-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 3.9 MB/s eta 0:00:00
     ━━━

In [61]:
!pip install --upgrade ipywidgets

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/b8/d4/ce436660098b2f456e2b8fdf76d4f33cbc3766c874c4aa2f772c7a5e943f/ipywidgets-8.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.7 from https://files.pythonhosted.org/packages/8e/d4/d31b12ac0b87e8cc9fdb6ea1eb6596de405eaaa2f25606aaa755d0eebbc0/widgetsnbextension-4.0.8-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.7 from https://files.pythonhosted.org/packages/74/5e/2475ac62faf2e342b2bf20b8d8e375f49400ecb38f52e4e0a7557eb1cedb/jupyterlab_widgets-3.0.8-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 4.5 MB/s eta 0:00:00a 0:00:01


In [62]:
!pip install --upgrade notebook

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for notebook from https://files.pythonhosted.org/packages/79/97/b803f9686fe25b40c8eca2d1ba440b2f878f23b87765d18fe93dd323f0b7/notebook-7.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for jupyter-server<3,>=2.4.0 from https://files.pythonhosted.org/packages/96/a2/b432812537beaf22a9dbc0d50cb62471e57ef90df42738675760fb3dce98/jupyter_server-2.7.3-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-server<3,>=2.22.1 from https://files.pythonhosted.org/packages/a7/0d/6d4eab0391bd4df1c43f308368d5e177b0fa8ac632267222a23b71317091/jupyterlab_server-2.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab<5,>=4.0.2 from https://files.pythonhosted.org/packages/71/a3/38b9d6492a393dcfdae9e82021655432a9fd6e8f4c03c30a7b55036c0d70/jupyterlab-4.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for anyio>=3.1.0 

In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

In [65]:
from ipywidgets import widgets
from IPython.display import display

In [37]:
df_dashboard = pd.read_csv("../data/MPS/MPS_MonthlyCrimeDashboard_OtherCrimeDatafy23-24_07.csv")
df_dashboard.head()

,Month_Year,Area Type,Borough_SNT,Area Name,Area Code,Crime Type,Crime Subtype,Measure,Financial Year,FY_FYIndex,Count
0,2023-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Domestic Abuse,Domestic Abuse,Offences,fy23-24,fy23-24_07,3
1,2023-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Domestic Abuse,Domestic Abuse,Outcomes,fy23-24,fy23-24_07,1
2,2023-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Domestic Abuse,Domestic Abuse Incidents,Statistics,fy23-24,fy23-24_07,6
3,2023-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Domestic Abuse,Domestic Abuse Violence with Injury,Offences,fy23-24,fy23-24_07,1
4,2023-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Hate crime,Faith Crime,Outcomes,fy23-24,fy23-24_07,1


In [38]:
df_dashboard.nunique()

Month_Year          4
Area Type           2
Borough_SNT       745
Area Name         735
Area Code         762
Crime Type          5
Crime Subtype      26
Measure             3
Financial Year      1
FY_FYIndex          1
Count             574
dtype: int64

In [39]:
df_dashboard.select_dtypes(include= 'number')

,Count
0,3
1,1
2,6
3,1
4,1
...,...
43442,3
43443,9
43444,107
43445,3


In [40]:
df_borough = pd.read_csv('../data/MPS/MPS Borough Level Crime (Historical).csv')
df_borough.rename(columns={'MajorText': 'CrimeType', 'MinorText': 'CrimeSubtype', 'LookUp_BoroughName': 'BoroughName'}, inplace=True)
df_borough.head()

,CrimeType,CrimeSubtype,BoroughName,201004,201005,201006,201007,201008,201009,201010,...,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107
0,Arson and Criminal Damage,Arson,Barking and Dagenham,6,5,11,10,6,6,13,...,7,4,2,4,6,4,6,6,5,4
1,Arson and Criminal Damage,Criminal Damage,Barking and Dagenham,208,193,227,220,204,163,199,...,120,100,110,100,104,80,99,128,141,130
2,Burglary,Burglary Business and Community,Barking and Dagenham,49,58,59,47,46,51,52,...,20,18,24,21,18,14,12,30,24,21
3,Burglary,Domestic Burglary,Barking and Dagenham,118,102,124,137,153,136,116,...,68,90,91,70,90,71,75,81,66,61
4,Drug Offences,Drug Trafficking,Barking and Dagenham,9,6,5,12,9,7,11,...,15,18,13,12,11,12,9,9,7,6


In [41]:
df_borough.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1725 entries, 0 to 1724
Columns: 139 entries, CrimeType to 202107
dtypes: int64(136), object(3)
memory usage: 1.8+ MB


In [42]:
df_borough.isnull().sum()

CrimeType       0
CrimeSubtype    0
BoroughName     0
201004          0
201005          0
               ..
202103          0
202104          0
202105          0
202106          0
202107          0
Length: 139, dtype: int64

In [43]:
df_borough[df_borough['CrimeSubtype'].str.contains('Other Theft')]

,CrimeType,CrimeSubtype,BoroughName,201004,201005,201006,201007,201008,201009,201010,...,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107
42,Theft,Other Theft,Barking and Dagenham,213,236,223,271,202,226,197,...,138,127,141,130,119,126,158,133,166,168
95,Theft,Other Theft,Barnet,297,304,332,320,291,335,342,...,221,235,228,166,142,181,216,221,269,221
145,Theft,Other Theft,Bexley,157,137,160,140,132,151,133,...,97,89,97,94,69,85,84,95,134,120
196,Theft,Other Theft,Brent,259,282,294,320,247,255,214,...,259,180,230,209,193,252,187,207,251,284
248,Theft,Other Theft,Bromley,265,334,299,311,312,295,304,...,143,129,148,119,147,131,138,158,169,181
300,Theft,Other Theft,Camden,729,695,617,666,676,656,756,...,407,223,261,160,147,184,196,279,406,391
353,Theft,Other Theft,Croydon,389,321,422,465,492,417,360,...,235,207,226,179,205,201,252,266,327,343
406,Theft,Other Theft,Ealing,368,422,412,443,368,322,387,...,271,210,191,188,167,211,222,244,276,222
460,Theft,Other Theft,Enfield,185,188,162,190,219,214,230,...,235,231,227,192,198,232,243,248,253,230
513,Theft,Other Theft,Greenwich,267,274,306,329,264,314,280,...,197,177,163,153,136,162,162,185,220,229


In [44]:
df_borough.nunique()

CrimeType        12
CrimeSubtype     56
BoroughName      33
201004          253
201005          262
               ... 
202103          230
202104          234
202105          239
202106          253
202107          253
Length: 139, dtype: int64

In [45]:
df_borough.columns

Index(['CrimeType', 'CrimeSubtype', 'BoroughName', '201004', '201005',
       '201006', '201007', '201008', '201009', '201010',
       ...
       '202010', '202011', '202012', '202101', '202102', '202103', '202104',
       '202105', '202106', '202107'],
      dtype='object', length=139)

In [47]:
long_df_all = pd.melt(df_borough, id_vars=['BoroughName', 'CrimeType', 'CrimeSubtype'], var_name='TimePeriod', value_name='Value')
long_df_all

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value
0,Barking and Dagenham,Arson and Criminal Damage,Arson,201004,6
1,Barking and Dagenham,Arson and Criminal Damage,Criminal Damage,201004,208
2,Barking and Dagenham,Burglary,Burglary Business and Community,201004,49
3,Barking and Dagenham,Burglary,Domestic Burglary,201004,118
4,Barking and Dagenham,Drug Offences,Drug Trafficking,201004,9
...,...,...,...,...,...
234595,Westminster,Vehicle Offences,Theft from a Motor Vehicle,202107,218
234596,Westminster,Vehicle Offences,Theft or Taking of a Motor Vehicle,202107,56
234597,Westminster,Violence Against the Person,Homicide,202107,1
234598,Westminster,Violence Against the Person,Violence with Injury,202107,416


In [48]:
long_df_all.sort_values(by='Value', ascending=False).head(50)

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value
201815,Westminster,Theft,Other Theft,201912,2265
188015,Westminster,Theft,Other Theft,201904,2165
36215,Westminster,Theft,Other Theft,201112,2111
189740,Westminster,Theft,Other Theft,201905,2102
191465,Westminster,Theft,Other Theft,201906,2046
41390,Westminster,Theft,Other Theft,201203,2000
193190,Westminster,Theft,Other Theft,201907,1985
198365,Westminster,Theft,Other Theft,201910,1982
200090,Westminster,Theft,Other Theft,201911,1978
186290,Westminster,Theft,Other Theft,201903,1962


In [49]:
long_df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234600 entries, 0 to 234599
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   BoroughName   234600 non-null  object
 1   CrimeType     234600 non-null  object
 2   CrimeSubtype  234600 non-null  object
 3   TimePeriod    234600 non-null  object
 4   Value         234600 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 8.9+ MB


In [50]:
sorted_theft_df = long_df_all[long_df_all['CrimeType'] == 'Theft'].sort_values(by='Value', ascending=False)
sorted_antisocial_df

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value
201815,Westminster,Theft,Other Theft,2019-12-01,2265
188015,Westminster,Theft,Other Theft,2019-04-01,2165
36215,Westminster,Theft,Other Theft,2011-12-01,2111
189740,Westminster,Theft,Other Theft,2019-05-01,2102
191465,Westminster,Theft,Other Theft,2019-06-01,2046
...,...,...,...,...,...
18443,London Heathrow and London City Airports,Theft,Theft from Person,2011-02-01,0
63290,London Heathrow and London City Airports,Theft,Bicycle Theft,2013-04-01,0
178865,London Heathrow and London City Airports,Theft,Bicycle Theft,2018-11-01,0
111590,London Heathrow and London City Airports,Theft,Bicycle Theft,2015-08-01,0


In [51]:
long_df_all['TimePeriod'] = pd.to_datetime(long_df_all['TimePeriod'], format='%Y%m')
long_df_all

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value
0,Barking and Dagenham,Arson and Criminal Damage,Arson,2010-04-01,6
1,Barking and Dagenham,Arson and Criminal Damage,Criminal Damage,2010-04-01,208
2,Barking and Dagenham,Burglary,Burglary Business and Community,2010-04-01,49
3,Barking and Dagenham,Burglary,Domestic Burglary,2010-04-01,118
4,Barking and Dagenham,Drug Offences,Drug Trafficking,2010-04-01,9
...,...,...,...,...,...
234595,Westminster,Vehicle Offences,Theft from a Motor Vehicle,2021-07-01,218
234596,Westminster,Vehicle Offences,Theft or Taking of a Motor Vehicle,2021-07-01,56
234597,Westminster,Violence Against the Person,Homicide,2021-07-01,1
234598,Westminster,Violence Against the Person,Violence with Injury,2021-07-01,416


In [52]:
agg_df_borough_2010 = df_borough.groupby(['BoroughName', 'CrimeSubtype'])[['201004', '201005', '201006', '201007', '201008', '201009', '201010', '201011', '201012']].sum()
agg_df_borough_2010.reset_index(inplace=True)
agg_df_borough_2010

,BoroughName,CrimeSubtype,201004,201005,201006,201007,201008,201009,201010,201011,201012
0,Barking and Dagenham,Absconding from Lawful Custody,0,0,0,0,0,0,0,0,0
1,Barking and Dagenham,Aggravated Vehicle Taking,1,1,2,1,3,2,0,1,0
2,Barking and Dagenham,Arson,6,5,11,10,6,6,13,13,14
3,Barking and Dagenham,Bail Offences,0,0,0,0,0,0,0,0,0
4,Barking and Dagenham,Bicycle Theft,13,19,36,41,42,24,18,16,10
...,...,...,...,...,...,...,...,...,...,...,...
1720,Westminster,Threat or Possession With Intent to Commit Cri...,0,2,0,3,1,5,1,0,1
1721,Westminster,Violence with Injury,271,283,289,328,288,270,319,287,279
1722,Westminster,Violence without Injury,295,322,356,346,312,316,297,280,264
1723,Westminster,Violent Disorder,2,0,1,0,1,0,1,22,36


In [53]:
agg_df_borough_2010.describe()

,201004,201005,201006,201007,201008,201009,201010,201011,201012
count,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000,1725.000000
mean,39.731014,40.697391,41.183188,41.926377,38.788986,39.284638,40.767536,39.843478,33.936232
std,86.700030,87.337174,87.050943,91.646399,84.988546,85.090627,90.977222,89.716225,79.700487
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000,3.000000,2.000000
75%,34.000000,35.000000,36.000000,37.000000,35.000000,36.000000,36.000000,32.000000,25.000000
max,1689.000000,1616.000000,1547.000000,1807.000000,1563.000000,1632.000000,1892.000000,1798.000000,1812.000000


In [54]:
long_agg_df_2010 = pd.melt(agg_df_borough_2010, id_vars=['BoroughName', 'CrimeSubtype'], var_name='TimePeriod', value_name='Value')
print(long_agg_df_2010)

                BoroughName  \
0      Barking and Dagenham   
1      Barking and Dagenham   
2      Barking and Dagenham   
3      Barking and Dagenham   
4      Barking and Dagenham   
...                     ...   
15520           Westminster   
15521           Westminster   
15522           Westminster   
15523           Westminster   
15524           Westminster   

                                            CrimeSubtype TimePeriod  Value  
0                         Absconding from Lawful Custody     201004      0  
1                              Aggravated Vehicle Taking     201004      1  
2                                                  Arson     201004      6  
3                                          Bail Offences     201004      0  
4                                          Bicycle Theft     201004     13  
...                                                  ...        ...    ...  
15520  Threat or Possession With Intent to Commit Cri...     201012      1  
15521      

In [55]:
long_agg_df_2010.sort_values(by="Value", ascending=False).head(50)

,BoroughName,CrimeSubtype,TimePeriod,Value
12050,Westminster,Other Theft,201010,1892
15500,Westminster,Other Theft,201012,1812
6875,Westminster,Other Theft,201007,1807
13775,Westminster,Other Theft,201011,1798
1700,Westminster,Other Theft,201004,1689
10325,Westminster,Other Theft,201009,1632
3425,Westminster,Other Theft,201005,1616
8600,Westminster,Other Theft,201008,1563
5150,Westminster,Other Theft,201006,1547
12360,Camden,Other Theft,201011,920


In [66]:
profile = ProfileReport(long_df_all, title="Profiling Report")
profile.to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]